<a href="https://colab.research.google.com/github/TAMIDSpiyalong/Introduction-to-Machine-Learning-for-Energy/blob/main/Lecture_4a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Tokenziation and Vectorization

Text tokenization is a crucial step in NLP that involves reformatting a piece of text into smaller units called “tokens.” These tokens serve as the building blocks for text vectorization, which converts text into numerical representations (vectors) that machine learning models can work with.

## Objectives

1. Use the bag of words method to process text and compare the difference between texts.
2. Learn about TF-IDF (Term Frequency-Inverse Document Frequency) and use it to search texts.
3. Use pretrained models to vectorize texts.

## Basic Bag-of-Words (BOW)

We want to build a basic bag-of-words (BOW) representation of our corpus. You can probably do this from scratch using dictionaries and lists (and maybe that's a good exercise). Fortunately, there are robust libraries which make it easy. We can use the scikit-learn **CountVectorizer** which takes a collection of text documents and creates a matrix of token counts:<br>
https://scikit-learn.org/stable/index.html<br>

In [ ]:
import spacy
from scipy import spatial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# A corpus of sentences.
corpus = [
  "Red Bull drops hint on F1 engine.",
  "Honda exits F1, leaving F1 partner Red Bull.",
  "Hamilton eyes record eighth F1 title.",
  "Aston Martin announces sponsor."
]

In [ ]:
vectorizer = CountVectorizer()

The *fit_transform* method does two things:
1. It learns a vocabulary dictionary from the corpus.
2. It returns a matrix where each row represents a document and each column represents a token (i.e. term).<br>

In [ ]:
bow = vectorizer.fit_transform(corpus)

We can take a look at the features and vocabulary dictionary. Notice the **CountVectorizer** took care of tokenization for us. It also removed punctuation and lower-cased everything.

In [ ]:
vectorizer.get_feature_names_out()

Each scentence is represented in a row and each column corresponds to the word listed in the vocabulary. This is a very primitive way to carry information but it does its job to some level. Note the word sequence is not considered in such way of embedding.

In [ ]:
for i in bow:
    print(i.toarray()[0])

## Cosine Similarity

Using the **spatial** package, which is a collection of spatial algorithms and data structures, we can measure how similar are these scentences to each other. To get the cosine *similarity*, we have to substract the distance from 1.<br>

In [ ]:
# The cosine method expects array_like inputs, so we need to generate
# arrays from our sparse matrix.
doc1_vs_doc2 = 1 - spatial.distance.cosine(bow[0].toarray()[0], bow[1].toarray()[0])
doc1_vs_doc3 = 1 - spatial.distance.cosine(bow[0].toarray()[0], bow[2].toarray()[0])
doc1_vs_doc4 = 1 - spatial.distance.cosine(bow[0].toarray()[0], bow[3].toarray()[0])

print(corpus)

print(f"Doc 1 vs Doc 2: {doc1_vs_doc2}")
print(f"Doc 1 vs Doc 3: {doc1_vs_doc3}")
print(f"Doc 1 vs Doc 4: {doc1_vs_doc4}")

# TF-IDF


The TF-IDF (Term Frequency-Inverse Document Frequency) helps in assessing the importance of a term in a document relative to a corpus of documents. The TF score for a term $t$ in a document $d$ is calculated as the product of the term frequency

$TF(t,d)=f_{t,d}$

and the IDF

$IDF(t,D) =ln \frac{N+1}{n_t+1}+1$

where $N$ is the total number of documents and $n_t$ is the numbner of document $t$ appears in. The final TF-IDF score is expressed as:

$s_t=TF(t,d)*IDF(t,D)$

This formulation is consistent with the scikit-learn package. There are many variations of this score, e.g., with nomalization or $log_{10}$.


In the previous case, the TF-IDF score of "f1" in document 1 is equal to $1*(ln(5/4)+1)$.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# A corpus of sentences.
corpus = [
  "Red Bull drops hint on F1 engine.",
  "Honda exits F1, leaving F1 partner Red Bull.",
  "Hamilton eyes record eighth F1 title.",
  "Aston Martin announces sponsor."
]

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(norm=None)

# Fit and transform the corpus
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

print(tfidf_matrix.shape)

TF-IDF scores for each term in each document

In [ ]:
import pandas as pd
pd.DataFrame(tfidf_matrix.toarray(), columns = tfidf_vectorizer.get_feature_names_out())

In [ ]:
cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

In [ ]:
# Calculate cosine similarity between the first document and the others
print("Cosine Similarity between first document and the others:")
for i, doc in enumerate(tfidf_matrix[1:]):
    print(f"Document {i+2}: {cosine_similarity(tfidf_matrix[0], tfidf_matrix[i+1])}")


## A little search engine with TF-IDF

The TF-IDF features can also be used to look up some articles, which is how roughly search engine works. This time around, rather than using a short toy corpus, let's use a larger dataset. scikit-learn has a **datasets** module with utilties to load datasets.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

The **datasets** module includes fetchers for each dataset in scikit-learn. For our purposes, we'll fetch only the posts from the *sci.space* topic, and skip on headers, footers, and quoting of other posts.
By default, the fetcher retrieves the *training* subset of the data only. If you don't know what that means, it'll become clear later in the course when we discuss modelling. For now, it doesn't matter for our purposes.

In [ ]:
corpus = fetch_20newsgroups(categories=['sci.space'],
                            remove=('headers', 'footers', 'quotes'))

In [ ]:
# Number of posts in our dataset.
len(corpus.data)

In [ ]:
# View first two posts.
print(corpus.data[:2][0])

## Creating TF-IDF features

SpaCy is a library designed in Python and provides easy-to-use interfaces for various NLP tasks such as tokenization, named entity recognition (NER), part-of-speech (POS) tagging, dependency parsing, and more.

In [ ]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Define a sample sentence
sentence = "Red Bull drops hint on F1 engine."

# Process the sentence using SpaCy
doc = nlp(sentence)

# Tokenize the sentence and print each token along with its POS tag
print("Tokenization and POS tagging:")
for token in doc:
    print(token.text, token.pos_)

The usage pattern is similar, but we pass a customized tokenizer to the TFIDF vectorizor to make the dataset smaller and more efficient.

In [ ]:
# If we want to use spaCy's tokenizer, we need to create a callback.
nlp = spacy.load('en_core_web_sm')

# We don't need named-entity recognition nor dependency parsing for
# this so these components are disabled. This will speed up the
# pipeline.
unwanted_pipes = ["ner", "parser"]

# For this exercise, we'll remove punctuation and spaces (which
# includes newlines), filter for tokens consisting of alphabetic
# characters, and return the lemma (which require POS tagging).
def spacy_tokenizer(doc):
  with nlp.disable_pipes(*unwanted_pipes):
    return [t.lemma_ for t in nlp(doc) if \
            not t.is_punct and \
            not t.is_space and \
            t.is_alpha]

In [ ]:
%%time
# Use the default settings of TfidfVectorizer.
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
features = vectorizer.fit_transform(corpus.data)

In [ ]:
# The number of unique tokens.
print(len(vectorizer.get_feature_names_out()))

In [ ]:
# The dimensions of our feature matrix. X rows (documents) by Y columns (tokens).
print(features.shape)

In [ ]:
# What the encoding of the first document looks like in sparse format.
print(features[:5,:5].toarray())

As we mentioned, there are TF-IDF variations out there and scikit-learn, among other things, adds **smoothing** (adds a one to the numerator and denominator in the IDF component), and normalizes by default. These can be disabled if desired using the *smooth_idf* and *norm* parameters respectively. See here for more information:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


## Querying the data

The similarity measuring techniques we learned previously can be used here in the same way. In effect, we can query our data using this sequence:
1. *Transform* our query using the same vocabulary from our *fit* step on our corpus.
2. Calculate the pairwise cosine similarities between each document in our corpus and our query.
3. Sort them in descending order by score.

In [ ]:
# Transform the query into a TF-IDF vector.
query = ["lunar orbit"]
query_tfidf = vectorizer.transform(query)

In [ ]:
query_tfidf.shape

In [ ]:
# Calculate the cosine similarities between the query and each document.
# We're calling flatten() here becaue cosine_similarity returns a list
# of lists and we just want a single list.
cosine_similarities = cosine_similarity(features, query_tfidf).flatten()

Now that we have our list of cosine similarities, we can use this utility function to return the indices of the top k documents with the highest cosine similarities.

In [ ]:
import numpy as np

# numpy's argsort() method returns a list of *indices* that
# would sort an array:
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
#
# The sort is ascending, but we want the largest k cosine_similarites
# at the bottom of the sort. So we negate k, and get the last k
# entries of the indices list in reverse order. There are faster
# ways to do this using things like argpartition but this is
# more succinct.
def top_k(arr, k):
  kth_largest = (k + 1) * -1
  return np.argsort(arr)[:kth_largest:-1]

In [ ]:
# So for our query above, these are the top five documents.
top_related_indices = top_k(cosine_similarities, 5)
print(top_related_indices)

In [ ]:
# Let's take a look at their respective cosine similarities.
print(cosine_similarities[top_related_indices])

In [ ]:
# Top match.
print(corpus.data[top_related_indices[0]])

In [ ]:
# Second-best match.
print(corpus.data[top_related_indices[1]])

In [ ]:
# Try a different query
query = ["American company"]
query_tfidf = vectorizer.transform(query)

cosine_similarities = cosine_similarity(features, query_tfidf).flatten()
top_related_indices = top_k(cosine_similarities, 5)

print(top_related_indices)
print(cosine_similarities[top_related_indices])

In [ ]:
print(corpus.data[top_related_indices[1]])

So here we have the beginnings of a simple search engine but we're a far cry from competing with commercial off-the-shelf search engines, let alone Google.
<br>
- For each query, we're scanning through our entire corpus, but in practice, you'll want to create an **inverted index**. Search applications such as Elasticsearch do that under the hood.
- You'd also want to evaluate the efficacy of your search using metrics like **precision** and **recall**.
- Document ranking also tends to be more sophisticated, using different ranking functions like Okapi BM25. With major search engines, ranking also involves hundreds of variables such as what the user searched for previously, what do they tend to click on, where are they physically, and on and on. These variables are part of the "secret sauce" and are closely guarded by companies.
- Beyond word presence, intent and meaning are playing a larger role.
<br>

Information Retrieval is a huge, rich topic and beyond search, it's also key in tasks such as question-answering.

# Using Pretrained, Third-Party Vectors
There are a variety of pretrained, static word vector packages out there. In this section, we'll use the **Google News** vectors, a collection of three million, 300-dimension word vectors trained from three billion words from a Google News corpus (circa 2015).

In [ ]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import spacy
import tensorflow as tf

from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
!gdown "https://drive.google.com/uc?id=1BpfbHu4denceXiv8yfdY3EHgjKIcULku"

In [ ]:
embedding_file = 'GoogleNews-vectors-negative300.bin.gz'

Next, we'll have **gensim** load the vectors through the **KeyedVectors** module which will enable us to look up vectors by tokens and indices.<br>
https://radimrehurek.com/gensim/models/keyedvectors.html
<br><br>
To save time and space, we'll limit ourselves to 200,000 word vectors for now.

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format(embedding_file, binary=True, limit=200000)

In [ ]:
len(word_vectors)

In [ ]:
pizza = word_vectors['pizza']
print(f'Vector dimension: {pizza.shape}')

# The embedding for the word 'pizza'.
print(pizza)


In [ ]:
print(word_vectors.similarity('pizza', 'gorilla'))
print(word_vectors.similarity('pizza', 'tree'))
print(word_vectors.similarity('pizza', 'yoga'))
print(word_vectors.similarity('pizza', 'tomato'))
print(word_vectors.similarity('pizza', 'sauce'))
print(word_vectors.similarity('pizza', 'cheese'))

In [ ]:
word_vectors.n_similarity("dog bites man".split(), "canine nips human".split())


Visualizing word vectors is straight-forward and can offer insights into what kind of contexts the training algorithm picked up. Because these word vectors have a dimension of 300, we need to reduce them down to two dimensions to plot them on a regular graph. This can be done through **Principal Components Analysis (PCA)**. Here, we're plotting the words we considered in the slides.

In [ ]:
def display_pca_scatterplot(model, words):
    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]

    plt.figure(figsize=(10,10))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r', s=128)
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

display_pca_scatterplot(word_vectors, ['swim', 'swimming', 'cat', 'dog', 'feline', 'road', 'car', 'bus'])

## Conclustion

In this NLP lab, we delved into text tokenization and vectorization, starting with bag-of-words (BOW) representation for converting text into numerical forms using scikit-learn, then progressing to TF-IDF for assessing term importance in documents relative to a corpus. We utilized TF-IDF for text search and semantic analysis via cosine similarity metrics. Implementing a basic search engine using TF-IDF showcased practical application. Additionally, we explored pre-trained word vectors like Google News vectors, demonstrating how third-party resources can enhance NLP tasks.